<a href="https://colab.research.google.com/github/hsiangwei0903/Fine-Grained-Object-Recognition/blob/dev%2Fravi%2Ftent/Notebooks/EfficientNetB0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ravi Sangani**

EfficientNet is a convolutional neural network architecture and scaling method that uniformly scales all dimensions of depth/width/resolution using a compound coefficient. Unlike conventional practice that arbitrary scales these factors, the EfficientNet scaling method uniformly scales network width, depth, and resolution with a set of fixed scaling coefficients. For example, if we want to use $2^N$ times more computational resources, then we can simply increase the network depth by $\alpha ^ N$, width by $\beta ^ N$, and image size by $\gamma ^ N$, where $\alpha, \beta, \gamma$ are constant coefficients determined by a small grid search on the original small model. EfficientNet uses a compound coefficient $\phi$ to uniformly scales network width, depth, and resolution in a principled way.

The compound scaling method is justified by the intuition that if the input image is bigger, then the network needs more layers to increase the receptive field and more channels to capture more fine-grained patterns on the bigger image.

The base EfficientNet-B0 network is based on the inverted bottleneck residual blocks of MobileNetV2, in addition to squeeze-and-excitation blocks.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install timm


Mounted at /content/drive
     |████████████████████████████████| 431 kB 5.2 MB/s 


In [ ]:
cd /content/drive/My Drive/UW/2021-22_Senior/Capstone/ENGINE: Wyze/Images/test_yt/n02085620-Chihuahua

/content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_google/n02085620-Chihuahua


## **running inference on single image**

In [ ]:
import timm
import time
model = timm.create_model('efficientnet_b0', pretrained=True)
model.eval()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth


EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
   

In [ ]:
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print(config)

# url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
# urllib.request.urlretrieve(url, filename)
# img = Image.open(filename).convert('RGB')
filename = "n02085620_0.jpg"

img = Image.open(filename).convert('RGB')
read_image = mpimg.imread(filename)
implot = plt.imshow(read_image)
plt.show()

tensor = transform(img).unsqueeze(0) # transform and add batch dimension
plt.imshow(np.transpose(transform(img)))
plt.show()



{'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875}


FileNotFoundError: ignored

In [ ]:
import torch
with torch.no_grad():
  out = model(tensor)
probabilities = torch.nn.functional.softmax(out[0], dim=0)
print(probabilities.shape)

torch.Size([1000])


In [ ]:
# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

picture_of = "Italian greyhound"
print([picture_of in categories])
# Print top categories per image
top10_prob, top10_catid = torch.topk(probabilities, 10)
for i in range(top10_prob.size(0)):
    print(categories[top10_catid[i]], top10_prob[i].item())

In [ ]:
TOTAL_IMAGES_ENTIRE = 0;
TOTAL_CORRECT_ENTIRE = 0;

## **running inferences on all ratterrier/chihuahua images**
-avg latency is calculated here too


In [ ]:
cd /content/drive/My Drive/UW/2021-22_Senior/Capstone/ENGINE: Wyze/Images/test_yt/n02085620-Chihuahua

/content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02085620-Chihuahua


In [ ]:
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch
import os
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print(config)

accurate_prediction_counter = 0
total_number_of_images = 0
latency_sum = 0
# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
categories = [category.lower() for category in categories]
picture_of = "chihuahua"
key_words = ["chihuahua"]
softmax_accuracy_threshold = 0.0
contains = (picture_of in categories)
print(picture_of + " is a category in ImageNet?: " + str(contains))

for filename in os.listdir("."):
  latency_s = time.time()
  if filename.endswith("jpg"):
    total_number_of_images += 1
    img = Image.open(filename).convert('RGB')
    tensor = transform(img).unsqueeze(0) # transform and add batch dimension
    with torch.no_grad():
      out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    latency_curr_time = (time.time() - latency_s) # find latency end
    latency_sum += latency_curr_time
    accurate_prediction = False
    # Print top categories per image
    top3_prob, top3_catid = torch.topk(probabilities, 1)
    for i in range(top3_prob.size(0)):
      # print(categories[top3_catid[i]], top3_prob[i].item())
      category = categories[top3_catid[i]]
      for key_word in key_words:
        if (key_word in category) and (top3_prob[i].item() > softmax_accuracy_threshold):
          accurate_prediction = True
    if (accurate_prediction):
      # print(filename + " had its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      # print("\n")
      accurate_prediction_counter += 1
    # else:
    #   print(filename + " did not have its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
    #   print("\n")
accuracy = (accurate_prediction_counter / total_number_of_images) * 100
print("Total images: " + str(total_number_of_images))
print("EfficientNetB0121 had a " + str(accuracy) + "% accuracy on images in " + os.getcwd())
average_time = (latency_sum)/total_number_of_images
print("Average latency is " + str(average_time))
TOTAL_IMAGES_ENTIRE += total_number_of_images
TOTAL_CORRECT_ENTIRE += accurate_prediction_counter



{'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875}
chihuahua is a category in ImageNet?: True
Total images: 10
EfficientNetB0121 had a 60.0% accuracy on images in /content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02085620-Chihuahua
Average latency is 0.41196508407592775


## **running inferences on all husky images**

In [ ]:
cd /content/drive/My Drive/UW/2021-22_Senior/Capstone/ENGINE: Wyze/Images/test_yt/n02110185-Siberian_husky

/content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02110185-Siberian_husky


In [ ]:
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch
import os
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print(config)

accurate_prediction_counter = 0
total_number_of_images = 0

# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
categories = [category.lower() for category in categories]
picture_of = "siberian husky"
key_words = ["husky"]
softmax_accuracy_threshold = 0.0
contains = (picture_of in categories)
print(picture_of + " is a category in ImageNet?: " + str(contains))
print("\n")

for filename in os.listdir("."):
  if filename.endswith("jpg"):
    total_number_of_images += 1
    img = Image.open(filename).convert('RGB')
    tensor = transform(img).unsqueeze(0) # transform and add batch dimension
    with torch.no_grad():
      out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    accurate_prediction = False
    # Print top categories per image
    top3_prob, top3_catid = torch.topk(probabilities, 1)
    for i in range(top3_prob.size(0)):
      print(categories[top3_catid[i]], top3_prob[i].item())
      category = categories[top3_catid[i]]
      for key_word in key_words:
        if (key_word in category) and (top3_prob[i].item() > softmax_accuracy_threshold):
          accurate_prediction = True
    if (accurate_prediction):
      print(filename + " had its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
      accurate_prediction_counter += 1
    else:
      print(filename + " did not have its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
accuracy = (accurate_prediction_counter / total_number_of_images) * 100
print("Total images: " + str(total_number_of_images))
print("EfficientNetB0121 had a " + str(accuracy) + "% accuracy on images in " + os.getcwd())
TOTAL_IMAGES_ENTIRE += total_number_of_images
TOTAL_CORRECT_ENTIRE += accurate_prediction_counter



{'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875}
siberian husky is a category in ImageNet?: True


siberian husky 0.8132025599479675
n02110185_0.jpg had its actual category in the top 1 prediction with softmax > 0.0


siberian husky 0.7214768528938293
n02110185_1.jpg had its actual category in the top 1 prediction with softmax > 0.0


siberian husky 0.41881847381591797
n02110185_2.jpg had its actual category in the top 1 prediction with softmax > 0.0


siberian husky 0.35797983407974243
n02110185_3.jpg had its actual category in the top 1 prediction with softmax > 0.0


eskimo dog 0.18752485513687134
n02110185_4.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


siberian husky 0.6738320589065552
n02110185_5.jpg had its actual category in the top 1 prediction with softmax > 0.0


eskimo dog 0.21567073464393616
n02110185_6.jpg did not have its actual category in the top

## **running inferences on all labrador images**

In [ ]:
cd /content/drive/My Drive/UW/2021-22_Senior/Capstone/ENGINE: Wyze/Images/test_yt/n02099712-Labrador_retriever

/content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02099712-Labrador_retriever


In [ ]:
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch
import os
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print(config)

accurate_prediction_counter = 0
total_number_of_images = 0

# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
categories = [category.lower() for category in categories]
picture_of = "labrador retriever"
key_words = ["labrador"]
softmax_accuracy_threshold = 0.0
contains = (picture_of in categories)
print(picture_of + " is a category in ImageNet?: " + str(contains))
print("\n")

for filename in os.listdir("."):
  if filename.endswith("jpg"):
    total_number_of_images += 1
    img = Image.open(filename).convert('RGB')
    tensor = transform(img).unsqueeze(0) # transform and add batch dimension
    with torch.no_grad():
      out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    accurate_prediction = False
    # Print top categories per image
    top3_prob, top3_catid = torch.topk(probabilities, 1)
    for i in range(top3_prob.size(0)):
      print(categories[top3_catid[i]], top3_prob[i].item())
      category = categories[top3_catid[i]]
      for key_word in key_words:
        if (key_word in category) and (top3_prob[i].item() > softmax_accuracy_threshold):
          accurate_prediction = True
    if (accurate_prediction):
      print(filename + " had its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
      accurate_prediction_counter += 1
    else:
      print(filename + " did not have its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
accuracy = (accurate_prediction_counter / total_number_of_images) * 100
print("Total images: " + str(total_number_of_images))
print("EfficientNetB0121 had a " + str(accuracy) + "% accuracy on images in " + os.getcwd())
TOTAL_IMAGES_ENTIRE += total_number_of_images
TOTAL_CORRECT_ENTIRE += accurate_prediction_counter



{'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875}
labrador retriever is a category in ImageNet?: True


rhodesian ridgeback 0.5315908193588257
n02099712_0.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


kuvasz 0.19253641366958618
n02099712_1.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


kelpie 0.20498140156269073
n02099712_2.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


airedale 0.1534002721309662
n02099712_3.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


irish setter 0.40218397974967957
n02099712_4.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


golden retriever 0.5995801091194153
n02099712_5.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


great pyrenees 0.3682955801486969
n02099712_6.jpg did 

## **running inferences on all greyhound images**

In [ ]:
cd /content/drive/My Drive/UW/2021-22_Senior/Capstone/ENGINE: Wyze/Images/test_yt/n02091032-Italian_greyhound

/content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02091032-Italian_greyhound


In [ ]:
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch
import os
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print(config)

accurate_prediction_counter = 0
total_number_of_images = 0

# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
categories = [category.lower() for category in categories]
picture_of = "italian greyhound"
key_words = ["greyhound"]
softmax_accuracy_threshold = 0.0
contains = (picture_of in categories)
print(picture_of + " is a category in ImageNet?: " + str(contains))
print("\n")

for filename in os.listdir("."):
  if filename.endswith("jpg"):
    total_number_of_images += 1
    img = Image.open(filename).convert('RGB')
    tensor = transform(img).unsqueeze(0) # transform and add batch dimension
    with torch.no_grad():
      out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    accurate_prediction = False
    # Print top categories per image
    top3_prob, top3_catid = torch.topk(probabilities, 1)
    for i in range(top3_prob.size(0)):
      print(categories[top3_catid[i]], top3_prob[i].item())
      category = categories[top3_catid[i]]
      for key_word in key_words:
        if (key_word in category) and (top3_prob[i].item() > softmax_accuracy_threshold):
          accurate_prediction = True
    if (accurate_prediction):
      print(filename + " had its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
      accurate_prediction_counter += 1
    else:
      print(filename + " did not have its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
accuracy = (accurate_prediction_counter / total_number_of_images) * 100
print("Total images: " + str(total_number_of_images))
print("EfficientNetB0121 had a " + str(accuracy) + "% accuracy on images in " + os.getcwd())
TOTAL_IMAGES_ENTIRE += total_number_of_images
TOTAL_CORRECT_ENTIRE += accurate_prediction_counter


{'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875}
italian greyhound is a category in ImageNet?: True


italian greyhound 0.732166051864624
n02091032_0.jpg had its actual category in the top 1 prediction with softmax > 0.0


italian greyhound 0.49038368463516235
n02091032_1.jpg had its actual category in the top 1 prediction with softmax > 0.0


italian greyhound 0.6394500136375427
n02091032_2.jpg had its actual category in the top 1 prediction with softmax > 0.0


italian greyhound 0.3496498465538025
n02091032_3.jpg had its actual category in the top 1 prediction with softmax > 0.0


italian greyhound 0.6611074805259705
n02091032_4.jpg had its actual category in the top 1 prediction with softmax > 0.0


whippet 0.3790211081504822
n02091032_5.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


italian greyhound 0.31632018089294434
n02091032_6.jpg had its actual category 

## **running inferences on all german sheperd images**

In [ ]:
cd /content/drive/My Drive/UW/2021-22_Senior/Capstone/ENGINE: Wyze/Images/test_yt/n02106662-German_shepherd

/content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02106662-German_shepherd


In [ ]:
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch
import os
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print(config)

accurate_prediction_counter = 0
total_number_of_images = 0

# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
categories = [category.lower() for category in categories]
print(categories)
picture_of = "german shepherd"
key_words = ["german shepherd"]
softmax_accuracy_threshold = 0.0
contains = (picture_of in categories)
print(picture_of + " is a category in ImageNet?: " + str(contains))
print("\n")

for filename in os.listdir("."):
  if filename.endswith("jpg"):
    total_number_of_images += 1
    img = Image.open(filename).convert('RGB')
    tensor = transform(img).unsqueeze(0) # transform and add batch dimension
    with torch.no_grad():
      out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    accurate_prediction = False
    # Print top categories per image
    top3_prob, top3_catid = torch.topk(probabilities, 1)
    for i in range(top3_prob.size(0)):
      print(categories[top3_catid[i]], top3_prob[i].item())
      category = categories[top3_catid[i]]
      for key_word in key_words:
        if (key_word in category) and (top3_prob[i].item() > softmax_accuracy_threshold):
          accurate_prediction = True
    if (accurate_prediction):
      print(filename + " had its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
      accurate_prediction_counter += 1
    else:
      print(filename + " did not have its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
accuracy = (accurate_prediction_counter / total_number_of_images) * 100
print("Total images: " + str(total_number_of_images))
print("EfficientNetB0121 had a " + str(accuracy) + "% accuracy on images in " + os.getcwd())
TOTAL_IMAGES_ENTIRE += total_number_of_images
TOTAL_CORRECT_ENTIRE += accurate_prediction_counter

{'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875}
['tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead', 'electric ray', 'stingray', 'cock', 'hen', 'ostrich', 'brambling', 'goldfinch', 'house finch', 'junco', 'indigo bunting', 'robin', 'bulbul', 'jay', 'magpie', 'chickadee', 'water ouzel', 'kite', 'bald eagle', 'vulture', 'great grey owl', 'european fire salamander', 'common newt', 'eft', 'spotted salamander', 'axolotl', 'bullfrog', 'tree frog', 'tailed frog', 'loggerhead', 'leatherback turtle', 'mud turtle', 'terrapin', 'box turtle', 'banded gecko', 'common iguana', 'american chameleon', 'whiptail', 'agama', 'frilled lizard', 'alligator lizard', 'gila monster', 'green lizard', 'african chameleon', 'komodo dragon', 'african crocodile', 'american alligator', 'triceratops', 'thunder snake', 'ringneck snake', 'hognose snake', 'green snake', 'king snake', 'garter snake', 'water snake', 

## **running inferences on all malamute images**

In [ ]:
cd /content/drive/MyDrive/UW/2021-22_Senior/Capstone/ENGINE: Wyze/Images/test_yt/n02110063-malamute

/content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02110063-malamute


In [ ]:
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch
import os
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print(config)

accurate_prediction_counter = 0
total_number_of_images = 0

# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
categories = [category.lower() for category in categories]
picture_of = "malamute"
key_words = ["malamute"]
softmax_accuracy_threshold = 0.0
contains = (picture_of in categories)
print(picture_of + " is a category in ImageNet?: " + str(contains))
print("\n")

for filename in os.listdir("."):
  if filename.endswith("jpg"):
    total_number_of_images += 1
    img = Image.open(filename).convert('RGB')
    tensor = transform(img).unsqueeze(0) # transform and add batch dimension
    with torch.no_grad():
      out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    accurate_prediction = False
    # Print top categories per image
    top3_prob, top3_catid = torch.topk(probabilities, 1)
    for i in range(top3_prob.size(0)):
      print(categories[top3_catid[i]], top3_prob[i].item())
      category = categories[top3_catid[i]]
      for key_word in key_words:
        if (key_word in category) and (top3_prob[i].item() > softmax_accuracy_threshold):
          accurate_prediction = True
    if (accurate_prediction):
      print(filename + " had its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
      accurate_prediction_counter += 1
    else:
      print(filename + " did not have its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
accuracy = (accurate_prediction_counter / total_number_of_images) * 100
print("Total images: " + str(total_number_of_images))
print("EfficientNetB0121 had a " + str(accuracy) + "% accuracy on images in " + os.getcwd())
TOTAL_IMAGES_ENTIRE += total_number_of_images
TOTAL_CORRECT_ENTIRE += accurate_prediction_counter

{'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875}
malamute is a category in ImageNet?: True


malamute 0.8728640079498291
n02110063_0.jpg had its actual category in the top 1 prediction with softmax > 0.0


eskimo dog 0.36664658784866333
n02110063_1.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


malamute 0.5233763456344604
n02110063_2.jpg had its actual category in the top 1 prediction with softmax > 0.0


malamute 0.42292413115501404
n02110063_3.jpg had its actual category in the top 1 prediction with softmax > 0.0


keeshond 0.15274477005004883
n02110063_4.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


malamute 0.536411464214325
n02110063_5.jpg had its actual category in the top 1 prediction with softmax > 0.0


malamute 0.3682514429092407
n02110063_6.jpg had its actual category in the top 1 prediction with softmax > 0.0


malam

### notes

[link text](https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F47%2F2021%2F08%2F10%2Falaskan-malamute-vs-siberian-husky-infographic.png
)

interesting results, the siberian husky is almost identical to malamute


## running inference on all miniature schnauzer images
-avg throughput per 1 min is also calculated here

In [ ]:
cd /content/drive/My Drive/UW/2021-22_Senior/Capstone/ENGINE: Wyze/Images/test_yt/n02097047-miniature_schnauzer

/content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02097047-miniature_schnauzer


In [ ]:
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch
import os
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print(config)

accurate_prediction_counter = 0
total_number_of_images = 0

# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
categories = [category.lower() for category in categories]
picture_of = "miniature schnauzer"
key_words = ["miniature schnauzer"]
softmax_accuracy_threshold = 0.0
contains = (picture_of in categories)
print(picture_of + " is a category in ImageNet?: " + str(contains))
print("\n")

beginning_time = time.time()
for filename in os.listdir("."):
  if filename.endswith("jpg"):
    total_number_of_images += 1
    img = Image.open(filename).convert('RGB')
    tensor = transform(img).unsqueeze(0) # transform and add batch dimension
    with torch.no_grad():
      out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    accurate_prediction = False
    # Print top categories per image
    top3_prob, top3_catid = torch.topk(probabilities, 1)
    for i in range(top3_prob.size(0)):
      # print(categories[top3_catid[i]], top3_prob[i].item())
      category = categories[top3_catid[i]]
      for key_word in key_words:
        if (key_word in category) and (top3_prob[i].item() > softmax_accuracy_threshold):
          accurate_prediction = True
    if (accurate_prediction):
      # print(filename + " had its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      # print("\n")
      accurate_prediction_counter += 1
    # else:
    #   print(filename + " did not have its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
    #   print("\n")
ending_time = time.time()
accuracy = (accurate_prediction_counter / total_number_of_images) * 100
print("Total images: " + str(total_number_of_images))
print("EfficientNetB0121 had a " + str(accuracy) + "% accuracy on images in " + os.getcwd())
print(str(total_number_of_images) + " took this long: " + str(ending_time - beginning_time))
minute_throughput = (60 * total_number_of_images) / (ending_time - beginning_time)
print ("Minute throughpout: " + str(minute_throughput))
TOTAL_IMAGES_ENTIRE += total_number_of_images
TOTAL_CORRECT_ENTIRE += accurate_prediction_counter

{'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875}
miniature schnauzer is a category in ImageNet?: True


Total images: 10
EfficientNetB0121 had a 20.0% accuracy on images in /content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02097047-miniature_schnauzer
10 took this long: 4.010533094406128
Minute throughpout: 149.6060463475235


## running inference on all Doberman images

In [ ]:
cd /content/drive/My Drive/UW/2021-22_Senior/Capstone/ENGINE: Wyze/Images/test_yt/n02107142-Doberman

/content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02107142-Doberman


In [ ]:
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch
import os
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print(config)

accurate_prediction_counter = 0
total_number_of_images = 0

# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
categories = [category.lower() for category in categories]
picture_of = "doberman"
key_words = ["doberman"]
softmax_accuracy_threshold = 0.0
contains = (picture_of in categories)
print(picture_of + " is a category in ImageNet?: " + str(contains))
print("\n")

for filename in os.listdir("."):
  if filename.endswith("jpg"):
    total_number_of_images += 1
    img = Image.open(filename).convert('RGB')
    tensor = transform(img).unsqueeze(0) # transform and add batch dimension
    with torch.no_grad():
      out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    accurate_prediction = False
    # Print top categories per image
    top3_prob, top3_catid = torch.topk(probabilities, 1)
    for i in range(top3_prob.size(0)):
      print(categories[top3_catid[i]], top3_prob[i].item())
      category = categories[top3_catid[i]]
      for key_word in key_words:
        if (key_word in category) and (top3_prob[i].item() > softmax_accuracy_threshold):
          accurate_prediction = True
    if (accurate_prediction):
      print(filename + " had its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
      accurate_prediction_counter += 1
    else:
      print(filename + " did not have its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
accuracy = (accurate_prediction_counter / total_number_of_images) * 100
print("Total images: " + str(total_number_of_images))
print("EfficientNetB0121 had a " + str(accuracy) + "% accuracy on images in " + os.getcwd())
TOTAL_IMAGES_ENTIRE += total_number_of_images
TOTAL_CORRECT_ENTIRE += accurate_prediction_counter

{'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875}
doberman is a category in ImageNet?: True


doberman 0.5514718294143677
n02107142_0.jpg had its actual category in the top 1 prediction with softmax > 0.0


doberman 0.14994113147258759
n02107142_1.jpg had its actual category in the top 1 prediction with softmax > 0.0


doberman 0.9583460688591003
n02107142_2.jpg had its actual category in the top 1 prediction with softmax > 0.0


doberman 0.8576943278312683
n02107142_3.jpg had its actual category in the top 1 prediction with softmax > 0.0


doberman 0.5287362337112427
n02107142_4.jpg had its actual category in the top 1 prediction with softmax > 0.0


vizsla 0.3547234833240509
n02107142_5.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


doberman 0.5537109971046448
n02107142_6.jpg had its actual category in the top 1 prediction with softmax > 0.0


doberman 0.50324237

## running inference on all american staffordshire terrier images

In [ ]:
cd /content/drive/My Drive/UW/2021-22_Senior/Capstone/ENGINE: Wyze/Images/test_yt/n02093428-American_Staffordshire_terrier

/content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02093428-American_Staffordshire_terrier


In [ ]:
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch
import os
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print(config)

accurate_prediction_counter = 0
total_number_of_images = 0

# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
categories = [category.lower() for category in categories]
picture_of = "terrier"
key_words = ["terrier"]
softmax_accuracy_threshold = 0.0
contains = (picture_of in categories)
print(picture_of + " is a category in ImageNet?: " + str(contains))
print("\n")

for filename in os.listdir("."):
  if filename.endswith("jpg"):
    total_number_of_images += 1
    img = Image.open(filename).convert('RGB')
    tensor = transform(img).unsqueeze(0) # transform and add batch dimension
    with torch.no_grad():
      out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    accurate_prediction = False
    # Print top categories per image
    top3_prob, top3_catid = torch.topk(probabilities, 1)
    for i in range(top3_prob.size(0)):
      print(categories[top3_catid[i]], top3_prob[i].item())
      category = categories[top3_catid[i]]
      for key_word in key_words:
        if (key_word in category) and (top3_prob[i].item() > softmax_accuracy_threshold):
          accurate_prediction = True
    if (accurate_prediction):
      print(filename + " had its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
      accurate_prediction_counter += 1
    else:
      print(filename + " did not have its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
accuracy = (accurate_prediction_counter / total_number_of_images) * 100
print("Total images: " + str(total_number_of_images))
print("EfficientNetB0121 had a " + str(accuracy) + "% accuracy on images in " + os.getcwd())
TOTAL_IMAGES_ENTIRE += total_number_of_images
TOTAL_CORRECT_ENTIRE += accurate_prediction_counter

{'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875}
terrier is a category in ImageNet?: False


chihuahua 0.5648274421691895
n02093428_0.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


staffordshire bullterrier 0.5088037252426147
n02093428_1.jpg had its actual category in the top 1 prediction with softmax > 0.0


american staffordshire terrier 0.433464914560318
n02093428_2.jpg had its actual category in the top 1 prediction with softmax > 0.0


american staffordshire terrier 0.9101479649543762
n02093428_3.jpg had its actual category in the top 1 prediction with softmax > 0.0


Total images: 4
EfficientNetB0121 had a 75.0% accuracy on images in /content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02093428-American_Staffordshire_terrier


## running inference on all standard poodle images

In [ ]:
cd /content/drive/My Drive/UW/2021-22_Senior/Capstone/ENGINE: Wyze/Images/test_yt/n02113799-standard_poodle

/content/drive/.shortcut-targets-by-id/1beXLuTyfFnOo3t19p65wHXF-qaxghvjI/ENGINE: Wyze/Images/test_yt/n02113799-standard_poodle


In [ ]:
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch
import os
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print(config)

accurate_prediction_counter = 0
total_number_of_images = 0

# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
categories = [category.lower() for category in categories]
picture_of = "standard poodle"
key_words = ["poodle"]
softmax_accuracy_threshold = 0.0
contains = (picture_of in categories)
print(picture_of + " is a category in ImageNet?: " + str(contains))
print("\n")

for filename in os.listdir("."):
  if filename.endswith("jpg"):
    total_number_of_images += 1
    img = Image.open(filename).convert('RGB')
    tensor = transform(img).unsqueeze(0) # transform and add batch dimension
    with torch.no_grad():
      out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    accurate_prediction = False
    # Print top categories per image
    top3_prob, top3_catid = torch.topk(probabilities, 1)
    for i in range(top3_prob.size(0)):
      print(categories[top3_catid[i]], top3_prob[i].item())
      category = categories[top3_catid[i]]
      for key_word in key_words:
        if (key_word in category) and (top3_prob[i].item() > softmax_accuracy_threshold):
          accurate_prediction = True
    if (accurate_prediction):
      print(filename + " had its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
      accurate_prediction_counter += 1
    else:
      print(filename + " did not have its actual category in the top 1 prediction with softmax > " + str(softmax_accuracy_threshold))
      print("\n")
accuracy = (accurate_prediction_counter / total_number_of_images) * 100
print("Total images: " + str(total_number_of_images))
print("EfficientNetB0121 had a " + str(accuracy) + "% accuracy on images in " + os.getcwd())
TOTAL_IMAGES_ENTIRE += total_number_of_images
TOTAL_CORRECT_ENTIRE += accurate_prediction_counter

{'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875}
standard poodle is a category in ImageNet?: True


chow 0.343498557806015
n02113799_0.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


hyena 0.3403847813606262
n02113799_1.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


toy poodle 0.3280598819255829
n02113799_2.jpg had its actual category in the top 1 prediction with softmax > 0.0


standard poodle 0.42763248085975647
n02113799_3.jpg had its actual category in the top 1 prediction with softmax > 0.0


irish water spaniel 0.5190920829772949
n02113799_4.jpg did not have its actual category in the top 1 prediction with softmax > 0.0


standard poodle 0.2253575325012207
n02113799_5.jpg had its actual category in the top 1 prediction with softmax > 0.0


standard poodle 0.8908197283744812
n02113799_6.jpg had its actual category in the top 

In [ ]:
print("accuracy is " + str(TOTAL_CORRECT_ENTIRE/TOTAL_IMAGES_ENTIRE))
print("latency is: " + str(average_time))
print("throughput per 60s is " + str(minute_throughput))

accuracy is 0.5681818181818182
latency is: 0.41196508407592775
throughput per 60s is 149.6060463475235
